# Pacman with Reinforcement Learning

Brandon Johnson

brandonajohnson09@gmail.com

## Introduction

During my undergraduate degree, I was given the opportunity to work on a group project of my choice in a Software Engineering course. As most college students, we were interested in games and wanted to make something that would require a team effort. Immediately, I began thinking about recreating arcade games and suggested that we should do Pacman. The reason for this was simple, the game was easy enough for our group to complete on time, and I felt we could easily split the labor into making pacman, making the maze, and making the ghosts. Iknew that our project could (and would) be a great successs, but I did not know that this would start my obsession with artificial intelligence at the time.

Quickly, I became obsessed with how sophisticated the ghost movement was in the game. For something that was developed in the 1980s, there was an immense complexity to how the ghosts behaved. Even though the algorithms behind them were simple, I could distinguish between each ghosts movements and behavior. Each had its own personality and watching them come to life for the first time was one of the most satisfying moments in my programming career. With this said, I have now had a crash course in far more advanced artificial intelligence and would like to revisit the beginning of my journey. In order to do this, I thought I should focus on the other side of the game, Pacman.

Pacman is probably the perfect game for artificial intelligence. It has a finite world (given there are several states), the other agents that are involved have a pattern that can be learned, and it has a finite set of inputs. With this, I knew that I could solve this problem using reinforcement learning. I would able to construct keys for a Q dictionary with the state of the board and chose the best move for pacman accordingly.


For this project I chose to use the pygame library to create a GUI. pygame is available for free down at https://www.pygame.org/news

I used this reference for my initial project in Java. It breaks down exactly what each ghost is "thinking" http://gameinternals.com/post/2072558330/understanding-pac-man-ghost-behavior

I had planned on doing all of the ghosts and power pellets in a full game but quickly realized that this may not be doable with my current understanding of Q Learning and my lack of resources for running the millions of tests required to have a properly trained algorithm. I have seen this done before on YouTube, but I believe that it would require a highly optimized and non generic algorithm. My purpose here is to simply modify Dr. Anderson's implementation of Q Learning with Tic Tac Toe to a more complex game. This will open up a number of possibilities for extending the algorithm to more and more complex games and will hopefully further help me dive into the depths of artificial intelligence and game design. 

## Concept and Implementation

To represent the maze for the game, I chose to give each tile a key:


Pac Man      = *

Wall         = -

Dot          = .

Power Pellet = $

Empty Tile   = _

Warp Left    = L

Warp Right   = R

I have decided to represent the initial maze in a text file. After doing some quick analysis we have these totals:
868 tiles

568 walls

240 dots

52 empty tiles

4 power pellets

4 ghosts

2 warps

and

1 pac man

This gives us 240 + 52 + 4 + 2 + 1 + 1 = 300 walkable areas in the game. Note that only 1 ghost tile is accessible to Pac Man so I am only counting where Blinky starts at. Since each dot can turn into an empty tile that gives us * 240 = 72,000 + the possibility that each ghost and pac man can be in any of the 300 walkable areas as well. This quickly explodes to an unmanageable number of states. Because of this, I have to cut the number of states down drastically. I chose to make a smaller board 16x17, to cut the number of ghosts down to one (Blinky), and to remove all of the warps and power pellets. If I could make this work adequately then I would add the other factors in later or potentially just save this for another experiment on another day. Additionally, for state reduction I began thinking of what possible representations of the board would be valid. There where several failed version of my dictionary keys before I setting on the current iteration. Here is a rundown:

For my Q scoring system I decided to check a couple things. I wanted Pacman to obviously go after pellets so I gave him a +1 reinforcement for every pellet he would consume. Obviously, if he cleared the board I wanted to make that extremely beneficial so I gave him +100 for clearing all pellets and winning the game. I decided to give him the complete opposite if he died so I gave him -100 for dying. This is something that would be revised several times during the experiment.


My first attempt was simple enough: I was going to represent the state of the board as everything in the maze + the move. This was an astronomical disaster. The reason for this is that every single changing object (like a pellet being ate) made the board an entirely different state. This lead to an explosion of states that eventually lead to me receiving this very unfortunate feedback from Jupyter Notebook:

[W 03:20:07.132 NotebookApp] IOPub data rate exceeded.                                                                      The notebook server will temporarily stop sending output                                                                to the client in order to avoid crashing it.                                                                            To change this limit, set the config variable                                                                           `--NotebookApp.iopub_data_rate_limit`.   

I had too many states which was causing an explosion of memory usage so I decided that I needed to cutdown on the amount of states by changing what factors I considered. This lead to my second iteration.

For my second attempt at representing the board, I decided to only track three variables: Pacman's position, Blinky's position and the move that was taken on that turn. This also proved to be unsuccessful because the variation still occurred too rapidly. Blinky could be in any of the available states as could Pacman, so Pacman still had no idea of what the best approach was and he would go to a corner and keep turning back and forth in the corner until Blinky came and killed him.

The reason for this failure was I needed to adjust the scoring part of my Q learning. I decided to make several adjustments here. First, I chose add a decay for every move that Pacman did not eat a pellet. This is to discourage moving to tiles that have no benefit. I tweaked this a few times from a range of -0.00001 to -1 and eventually landed on -0.1. I also decided to change the reward for a eating a pellet. At first this was 3, but I felt that 1 would be easier to comprehend. Then, I made winning worth +50 and losing worth -50. 

With this new idea on what I should be scoring, I decided to revamp my state representation once again. At this iteration, I chose to make Pacman now consider his position, Blinky's position and the pellets consumed. Obviously this result was no better than the previously ones. Adding more things to the state is always a bad idea. The things that I was adding were also already part of the solution. Pacman already knows that Blinky is bad because he gets a negative reinforcement when he runs into him. Blinky's algorithm isn't random so Pacman does not need to explicitly know where Blinky is. The pellets are static, they are either in their location or already eaten, so Pacman does not need to actively know where they are for him to eat them. When he eats one he will know because of the positive reinforcement, and if he goes to a path that does not benefit him, he will also get a negative reinforcement.

Next I tried to use the moves that took place. I kept adding the moves at the previous state to a list and using that as the dictionary key. This approached was flawed from the start as Pacman could take so many turns that the number of states was infinite. I was at a loss for what to do at this point so I scrapped my previous ideas and tried to get back to the basics of what makes Q learning successful. The concepts of Q learning are simple: you have an agent (Pacman) who has a goal (eat all of the pellets). With this idea, I decided that the most finite set for my states was the set of positions for Pacman + the set of positions for Blinky + the current move. This gave me 16X17X16X17X4 = 295936 states. Obviously this is still an enormous state space, but it was no longer infinite. At this point I was ready to begin testing.


## Program setup

My experiment was simple. I wrote the game of Pacman relatively quickly. Some of the more important methods that I defined were:

### Helper functions
   - `printMaze(maze)`:  
   
   
   This function simply takes in a maze (numpy array from a text file) and prints it to the screen
 
 
   - `noWall(direct,maze,sprite)`:
   
   
    This function takes a direction, maze, and a sprite and determines if the sprite is going to collide with a wall if it moves in the direction in the maze.
  
   - `getOppositeDir(direct)`:
  
   returns the opposite direction of the specified direction
  
  
   - `getValidMoves(sprite, maze)`:
   
   returns a list of valid moves for a sprite within its current position in the maze by checking that the directions do not intersect with a wall
           
    
  - `initMaze(mazename, size, smartghosts,numghosts,ghostspositions)`:
   
  creates and returns pacman, ghosts, maze, background, displayheight, displaywidth. This requires the name of the file of the maze image and txt file to be the same (minus the extension). size is used for spacing between tiles, smartghosts is a parameter to determine what move function that the ghosts will use. numghosts is the number of ghosts to spawn in the game. ghostspositions are the positions that the ghosts will start at.
  
  
  - `showMaze(maze, screen, size)`:
   
  this function is used to display the maze in pygame. It will print each of the pellets.
           

  - `hasCollision(sprite,ghost)`:
   
  this functions checks if the sprite (Pacman) collides with a ghost


  - `updateMap(sprite, maze, ghosts)`:
   
  this functions updates the maze array with the new position of Pacman. It will also remove any of the pellets that Pacman has consumed and checks the status of the game


  - `checkMove(sprite,maze,ghosts)`:
       
  this function is used by the Q Learning algorithm to check what the benefit of performing a move was.


  - `printQ(Q)`:
       
  this functions prints the keys and values in the given Q dictionary
 
 
  - `stateMoveTuple(pacman,ghosts,move)`:
  
  this function creates a tuple for the key in the Q dictionary by combining Pacman's position, the ghosts' position and the current move
  
  
  - `playGame(Q,epsilonDecayFactor,learningRate,mazename,gamespeed,numGames,debug,graphics,smartghosts,numghosts,gpositions)`:
       
  this function plays the game.
  
   - Q: the Q dictionary to use
  
   - epsilonDecayFactor: the amount epsilon decays between games
   
   - learningRate: the amount of learning that occurs with each move
   
   - mazename: name of the maze file
   
   - gamespeed: speed that the graphical version of the game plays
   
   - numGames: number of games to play
   
   - debug: Boolean for printing debugging info
   
   - graphics: Boolean to determine whether to play the game in the background or with pygame
   
   - smartghosts: Boolean to determine ghost move function
   
   - numghosts: number of ghosts to spawn in the map
   
   - gpositions: starting ghost positions
           
### Move functions

Below are the move functions that have been implemented in the game. In an early iteration of my experiment I included a user move function, but removed it because I didn't want to spend time for the experiment debugging it.

  - `randomMove(sprite,maze)`:
   
  takes a random move from the valid moves for the sprite in the maze
  
  - `blinkyMove(sprite,maze)`:
 
  creates a list of moves to pass to `getBlinkyMoves`. This functions tries to avoid moving back and forth when possible
  
  - `getBlinkyMove(sprite,options,targX,targY,maze)`:
  
  takes a list of options and checks it's position against its target's position. Then determines which of the options will move closest to the target in a straight line.
  
  
  - `QMove(sprite, maze, Q, epsilon, ghosts)`:
   
  Q learning movement. This checks the Q dictionary for moves if a random number is less than the current epsilon. If the epsilon check passes, this function will call `getValidMoves` to get each possible move for the sprite, and then it will check the Q dictionary for these moves and returns the one with greatest value. If the epsilon check fails, it will return a random valid move.

### Moving Object Class

I implemented a class for this game. `MovingObject` represents each entity in the game that moves (Pacman and the ghosts).

This class has several methods and properties.

#### Methods

* `getScreenPosition(self)`:
    gets the objects position on the screen
    
* `getPosition(self)`:
    gets position in the maze

* `setPosition(self,x,y)`:
    sets the objects position in the maze
    
* `getImages(self)`: 
    returns list of images for the object
    
    
* `getImage(self,index)`
     returns the current image for the object
    
* `getAnimPos(self)`:
     returns the current frame of the object
    
* `updatePos(self)`:
     updates the objects position          
    
* `updateAnimPos(self,direct,maze)`:
     updates the objects animation frame
        
* `updatePrevPos(self)`:
     updates the objects previous position

* `getDir(self)`:
     returns the objects current direction
    
* `setDir(self,direction)`:
     sets the objects direction
        
* `move(self,maze, Q=None, epsilon=None, ghosts=None)`:
     checks if Q and epsilon were passed. If so, calls QMove. Otherwise, calls objects move function
        
* `setSpaceSymbol(self, maze)`:
     sets the space symbol
        
* `getScore(self)`:
     returns the objects score (Not in use)
    
* `setScore(self, score)`:
     sets the objects score (Not in use)
        
* `getSpaceSymbol(self)`:
     gets the current space symbol
    
* `getSymbol(self)`:
     returns the symbol for the object

#### Properties

* posX: x position in maze
* posY: y position in maze
* images: images for graphics
* anims: animation frames for images
* animPos: current animation frames
* framesBetween: frames between updating position in maze
* symbol: symbol in maze
* spacing: offset spacing in maze
* direction: direction that the object is moving
* spacesymbol: symbol that is underneath the object in the maze
* size: size of the object
* prevX: previous X position in maze, used to update previous space symbol
* prevY: previous Y position in maze, used to update previous space symbol
* score: current score (not implemented)




## Experiment


#### Learning Rate

In my experiment, Pacman's environment is completely observable. You can see every pellet, ghost and Pacman himself. Because of this, a high learning rate would be ideal. If I had added in Clyde, who is completely random in his movements, I would have used a lower learning rate because the randomness would lead to more of a stochastic environment. Since the environment is deterministic, I settled on 0.95 as a learning rate. This is because each move that Pacman does in relation to Blinky should be the same. If Blinky is in one position and Pacman is in another, Pacman should always chose to move to a position that does not allow Blinky to kill him or a position that wins the game.

#### Epsilon Decay Factor

This is the hardest part of the game. Because there are so many states, it becomes incredibly difficult to determine an appropriate decay factor for epsilon. If epsilon decays to quickly, Pacman is still essentially taking random moves because he has never encountered the moves that he is currently taking. If it decays to slowly the the randomness could negatively impact your overall Q value for a particular entry.I tried to make my decay factors work to where they would become zero as the game approaches its final iterations; however, I realized after running thousands and thousands of tests that it would be best to allow Pacman to play completely randomly for an extended period of time and then start to decay epsilon. Currently I am testing 200000 random iterations with no epsilon decay and then a decaying epsilon for another 50000 iterations. At this point, I will run the game with graphics to see what the results look like. In previous experiments, it seemed that Pacman did not learn quickly enough and would take the same losing moves over and over. I am hoping that more iterations will allow Pacman to see his environment more effectively.

#### Number of Trials

I have tested this for probably a 500000 trials now trying to figure out the exact parameters to use, but I have settled on about 200000-250000 completely random trials and then another 50000-100000 decaying trials. Then I will test my results with graphics and a few hundred trials. If this experiment does not yield the results I would like, I will attempt 500000 random trials and about 150000 decaying trials. 

#### Q Learning Algorithm

my Q learning algorithm is based on Assignment 5 from CS440 and Dr. Anderson's notes on Q Learning for Tic Tac Toe.


    get the current direction
    if the sprite can change direction (it is in the center of a tile: 0 for an animation frame)
        get a list of valid moves
        if random is l than epsilon
            get a random move
            set direction to the random move
        others
            Get the list of Q dictionary values from the valid moves
            set direction to the hightest Q value, if the value isn't in the list initial to 0
    
    if the current key is not in the Q dictionary add it
        
    update the sprites animations and movement
    return the direction and Q
    
    add the value of the current move to the current entry in the Q dictionary
    propogate this value back to the previous entry in the dictionary
    update the previous key to the current key
    
    
#### Results

My first several iterations of this experiment were colossal failures. I was previously adding the ghosts (Blinky) to the maze and trying to replace his symbol with the symbol he had ran over on the previous move, but I had several issues getting this to function properly. I chose instead to no longer represent the ghosts in the maze, and instead just keep their positions with their objects. Additionally, I attempted to give Pacman positive reinforcement for eating individual pellets, negative reinforcement for moving to tiles that give him no benefit, positive reinforcement for winning, and negative reinforcement for being killed, but I found out that these variables could skew the results in longer games. If Pacman ate several pellets then the result could outweigh dying or he could chose not to take a more beneficial path because a pellet did not exist there. 

To compensate for this, I chose to simplify my approach as much as possible. Only two states gave Pacman a reinforcement: winning and losing. This approach allowed me to focus only on keeping Pacman alive long enough to hit eat every pellet. I assumed that if he become very good at avoiding Blinky, eventually he would hit all points of the maze and clear the pellets. I feel that it would be more optimal to have the pellets and time taken actually factor into the score, but with my time crunch, I was unable to find a solution that adequately solved the problem.

I had the output of the non graphical playthroughs display at every 500 trials. This would display the number of games, number of wins, number of losses and current epsilon. This allowed me to see if I was making any progress or if I needed to make tweaks to the experiment and start over. Additionally, I printed the Q dictionary between each set of trials to determine if any extreme erroneous data had been accrued. In some early instance, I was getting values that were all 0's (not enough trials), negative infinity (over incrementing the Q values), and values in the 1000s (too many values being considered). These results kept me checking and tweaking my experiment very often.

The most unusual results that I had were trials that had the results converging to lose every match. Ironically, the average amount of wins for the random agent was about 20-30 in 500 or 5%, but an insufficiently trained agent will win a grand total of 0% of the time. This was due to several factors, but I suspect that my Q learning algorithm had some errors because it was subtracting the value of previous negative states and making them positive. I attempted to correct this, and hopefully the results of my final set of trials will be more correct.


For my results of 500,000 matches, it seemed that either the number of matches was insufficient to adequately represent the board, my key for the board was not precise enough to describe the situation, and/or my reinforcements were not precise enough to represent what is happening. I believe the reason for this is simple. Pacman needs to know more information that just his position and Blinky's position to be successful. If the only reinforcements Pacman is concerned with is staying away from Blinky then he is going to not be concerned with eating the pellets in the maze. If Pacman is concerned with eating pellets then he needs to know the locations of those pellets and whether or not they have been eaten. It makes no logical sense for Pacman to get a positive reinforcement for a key of P(12,1) B(12,2) L if he eats a pellet because the next time he comes to that spot, he will not receive the same positive reinforcement. This dilemma made this experiment go unsucessful to this point, and unfortunately the time constraints to run a million matches is making this experiment impossible to finish before the deadline. I attempted to use some shortcuts to make the results skewed to be much better. I modified the valid moves function to disallow moves in the opposite direction for random moves. This helped to win more random games and gave Pacman more of an idea of what was a good move to begin with. Unfortunately, I chose to allow those moves with the Q learning algorithm which causes Pacman to turn around abruptly at some points and die.

For a last ditch effort prior to the deadline. I chose to run 50,000 matches with no opposite moves. I think this will at least promote a better solution than what had occurred previously. I also chose to tweak the learningRate to be .75. I had previously experiment with rates as low as .2 and as high as .9, but with the amount of time left to test, I feel this is my only chance to have the results work properly. I added the item that is actually at that direction that Pacman is moving to the key. I think that this will hopefully provide more depth to Pacman's decisions and help to make the game find a positive resolution. Finally, for this iteration I chose to give the same benefit for clearing the board as eating a pellet 0.01. This means that the highest impact on Pacman is dying. There should be no random movements to a space with no benefit because Pacman should always be mindful of staying away from Blinky. The slight reinforcement for eating pellets should hopefully make Pacman want to move around the map instead of staying in one spot, and the disallowance of opposite moves will prevent wiggling back and forth.

The results of this experiment were unfortunately as expected. I did not have enough trials to properly train my Pacman. I have set the game now to run for 100,000,000 trials on its next run. I will turn in the project now, and I will eventually run it again on my own time. Hopefully, the additional training will provide better results. As for this iteration, Pacman would move towards pellets but would eventually be cut off by Blinky. I think that additional trials or tweaking the reinforcement and learning rate could prove to be beneficial.




#### What I Would Like to Test Next

I think that a neural network would be the best approach to solving this problem. You can represent all of the states without the massive amount of keys that would be required to represent every situation in Pacman. I also believe that I could potential look into splitting the map into several sectors and have the keys only represent a small area around Pacman. If I did this segment key it would look something like this

    P(X,Y) B(X,Y) PelletsByDirection Direction
    
I think this result may be more optimal because it would prevent at least some of the state explosion that would occur with listing the entire maze, and it would allow Pacman to make more wise decisions from his current position because he would know what his benefit would be from any valid direction to return. This could be something to help solve the problem more optimally and with better result. I would also like to have more time to test. Unfortunately, I was at crunch time still getting less than optimal results with a low epsilon and no results with 0 epsilon, so I was unable to test for as long as I would like to on the final iterations. If I had more time, I feel that I could have found the optimal solution to this problem.

## Conclusion



Pacman has always been one of my favorite games. Its not very complex on the surface level (4 ghosts, a few hundred pellets, 4 powerups, a maze, some fruit and 1 Pacman); however, it was really one of the first true innovations that we have had in artificial intelligence in mainstream gaming. Each ghost had its own movement algorithm, and it created something that seemed to run flawlessly and with some decision making. I wanted to faithfully recreate the game, but quickly realized that I would have invested far too much time in the gameplay of Pacman and not enough time on the real problem I was trying to solve. While I have not found the most appropriate solution to Pacman, I have learned a lot about reinforcement learning, and artificial intelligence in general. I feel that this new found knowledge will help to further my career as a programmer, and hopefully lead me to finding solutions to this problem and even more complex problems in the future. For this reason, I consider my attempt at the experiment a complete success, and even though the results were not what I expected, the process of watching my AI Pacman attempt to solve this problem was genuinely rewarding.

If I could leave any message for what this process meant to me, it would be to always try to do your best at anything that you do. Every problem has a solution, and it just takes a lot of dedication and thinking to solve it. Unfortunately, time constraints are a real concern for all of us, but if you are passionate about something, you should work towards it and acheive it. This is not going to be my last attempt at Pacman. I intend to continue to work on this project from time to time. I have added it to my github account. The project can be found here:  https://github.com/BrandonAJohnson15/reinforcement-learning-pacman

## Implementation

I am using the numpy loadtxt function to load a text file maze.txt that represents the maze. 

In [1]:
import sys, pygame
from pygame.locals import *
import numpy as np
import math
import random
import copy

useQ = True

In [2]:
def printMaze(maze):
    for i in range(len(maze)):
        for j in range(len(maze[0])):
            print(maze[i][j], end=" ")
        print()

In [3]:
printMaze(np.genfromtxt("small-maze.txt", dtype="str", delimiter=" "))

- - - - - - - - - - - - - - - - 
- . . . . . . - - . . . . . . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . . . . . . . . . . . . . . - 
- . - - - _ - - - - _ - - - . - 
- . - - - _ - - - - _ - - - . - 
- . - - - _ - - - - _ - - - . - 
- . - - - _ - - - - _ - - - . - 
- . - - - _ - - - - _ - - - . - 
- . . . . . . . * . . . . . . - 
- . - - - - - - - - - - - - . - 
- . - - - - - - - - - - - - . - 
- . . . . . . . . . . . . . . - 
- - - - - - - - - - - - - - - - 


In [4]:
printMaze(np.genfromtxt("maze.txt", dtype="str", delimiter=" "))

- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
- . . . . . . . . . . . . - - . . . . . . . . . . . . - 
- . - - - - . - - - - - . - - . - - - - - . - - - - . - 
- $ - - - - . - - - - - . - - . - - - - - . - - - - $ - 
- . - - - - . - - - - - . - - . - - - - - . - - - - . - 
- . . . . . . . . . . . . . . . . . . . . . . . . . . - 
- . - - - - . - - . - - - - - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - - - - - . - - . - - - - . - 
- . . . . . . - - . . . . - - . . . . - - . . . . . . - 
- - - - - - . - - - - - _ - - _ - - - - - . - - - - - - 
- - - - - - . - - - - - _ - - _ - - - - - . - - - - - - 
- - - - - - . - - _ _ _ _ B _ _ _ _ _ - - . - - - - - - 
- - - - - - . - - _ - - - - - - - - _ - - . - - - - - - 
- - - - - - . - - _ - _ _ _ _ _ _ - _ - - . - - - - - - 
L _ _ _ _ _ . _ _ _ - I _ P _ _ C - _ _ _ . _ _ _ _ _ R 
- - - - - - . - - _ - _ _ _ _ _ _ - _ - - . - - - - - - 
- - - - - - . - - _ - - - - - - - - _ - - . - - - - - - 
- - - - - - . - - _ _ _ _ _ _ _

In [5]:
class MovingObject:
    posX = 0
    posY = 0
    images = []
    anims = {}
    animPos = 0
    framesBetween = 0
    symbol = ""
    spacing = 0
    direction = ""
    spacesymbol = "_"
    size = 0
    prevX = -1
    prevY = -1
    score = 0
    
    def __init__(self, images, x, y, moveF, anims, framesBetween, symbol, spacing, size, direction):
        self.posX = x
        self.posY = y
        self.prevX = x
        self.prevY = y
        self.nextX = x
        self.nextY = y
        self.images = copy.copy(images)
        self.moveF = moveF
        self.anims = copy.deepcopy(anims)
        self.framesBetween = framesBetween
        self.symbol = symbol
        self.spacing = spacing
        self.size = size
        self.direction = direction
        
    def getScreenPosition(self):
        offsetX = -self.spacing
        offsetY = -self.spacing * 2
        if self.direction == 'L':
            offsetX -= self.spacing * self.animPos
        if self.direction == 'R':
            offsetX += self.spacing * self.animPos
        if self.direction == 'U':
            offsetY -= self.spacing * self.animPos
        if self.direction == 'D':
            offsetY += self.spacing * self.animPos
        return (self.posX*size+offsetX, self.posY*size+offsetY)
    
    def getPosition(self):
        return self.posX, self.posY
    
    def setPosition(self,x,y):
        self.posX = x
        self.posY = y
    
    def getImages(self):
        return self.images

    def getImage(self,index):
        if len(self.anims[self.direction]) > index:
            return self.images[self.anims[self.direction][index]]
        else:
            return self.images[self.anims[self.direction][0]]
    
    def getAnimPos(self):
        return self.animPos
    
    def updatePos(self):
        if self.animPos == 0:
            self.updatePrevPos()
            if self.direction == 'L':
                self.posX -= 1
            if self.direction == 'R':
                self.posX += 1
            if self.direction == 'U':
                self.posY -= 1
            if self.direction == 'D':
                self.posY += 1             
    
    def updateAnimPos(self,direct,maze):
        if direct == self.direction:
            self.animPos = (self.animPos + 1) % self.framesBetween
        elif self.animPos == 0:
            self.direction = direct
        else:
            if (direct == 'L' and self.direction == 'R' or direct == 'R' and self.direction == 'L' or
                direct == 'U' and self.direction == 'D' or direct == 'D' and self.direction == 'U'):
                self.animPos = (self.framesBetween - self.animPos) % self.framesBetween 
                self.direction = direct
        self.updatePos()
        
    def updatePrevPos(self):
        self.prevX = self.posX
        self.prevY = self.posY

    def getDir(self):
        return self.direction
    
    def setDir(self,direction):
        self.direction = direction
        
    def move(self,maze, Q=None, epsilon=None, ghosts=None):
        if not useQ or not Q and not epsilon:
            self.moveF(self,maze)
        else:
            return self.moveF(self,maze,Q,epsilon,ghosts)
        
    def setSpaceSymbol(self, maze):
        self.spacesymbol = maze[self.posY][self.posX]
        
    def getScore(self):
        return self.score
    
    def setScore(self, score):
        self.score = score
        
    def getSpaceSymbol(self):
        return self.spacesymbol
    
    def getSymbol(self):
        return self.symbol

In [6]:
def noWall(direct,maze,sprite):
    if direct == 'L':
        return (maze[sprite.posY][sprite.posX-1] != '-')
    if direct == 'R':
        return (maze[sprite.posY][sprite.posX+1] != '-')
    if direct == 'U':
        return (maze[sprite.posY-1][sprite.posX] != '-')
    if direct == 'D':
        return (maze[sprite.posY+1][sprite.posX] != '-')
    return False

In [7]:
def getOppositeDir(direct):
    if direct == 'L':
        return 'R'
    elif direct == 'R':
        return 'L'
    elif direct == 'U':
        return 'D'
    elif direct == 'D':
        return 'U'

In [8]:
def getValidMoves(sprite, maze, allowOpposite):
    dirs = ["L", "R", "U", "D"]
    options = []
    for d in dirs:
        if allowOpposite:
            if noWall(d,maze,sprite): 
                options.append(d)
        else:
            if noWall(d,maze,sprite) and getOppositeDir(sprite.getDir()) != d: 
                options.append(d)
    return options

In [9]:
def randomMove(sprite,maze):
    direct = sprite.getDir()
    if sprite.getAnimPos() == 0:
        options = getValidMoves(sprite, maze, False)
        if len(options) > 0:
            direct = options[random.randint(0,len(options)-1)]
    if (noWall(direct,maze,sprite)):        
        sprite.updateAnimPos(direct,maze)

In [10]:
def getBlinkyMove(sprite,options,targX,targY,maze):
    bestOptions = []
    if targX < sprite.getPosition()[0] and "L" in options:
        bestOptions.append("L")
    if targX > sprite.getPosition()[0] and "R" in options:
        bestOptions.append("R")
    if targY < sprite.getPosition()[1] and "U" in options:
        bestOptions.append("U")
    if targY > sprite.getPosition()[1] and "D" in options:
        bestOptions.append("D")
        
    if len(bestOptions) > 0:
        best = 0
        dist = 10000000
        for i in range(len(bestOptions)):
            if bestOptions[i] == "L" and sprite.getPosition()[0] - targX < dist:
                best = i
                dist = sprite.getPosition()[0] - targX
                if dist == 0:
                    dist += abs(targY - sprite.getPosition()[1])
            if bestOptions[i] == "R" and targX - sprite.getPosition()[0] < dist:
                best = i
                dist = targX - sprite.getPosition()[0]
                if dist == 0:
                    dist += abs(targY - sprite.getPosition()[1])
            if bestOptions[i] == "U" and targY - sprite.getPosition()[1] < dist:
                best = i
                dist = targY - sprite.getPosition()[1]
                if dist == 0:
                    dist += abs(targX - sprite.getPosition()[0])
            if bestOptions[i] == "D" and sprite.getPosition()[1] - targY < dist:
                best = i
                dist = sprite.getPosition()[1] - targY
                if dist == 0:
                    dist += abs(targX - sprite.getPosition()[0])
            # discourage from going back and forth
            if bestOptions[i] == getOppositeDir(sprite.getDir()):
                dist += 10              
        return bestOptions[best]
    else:
        return options[random.randint(0,len(options)-1)]

In [11]:
def blinkyMove(sprite,maze):
    direct = sprite.getDir()
    if sprite.getAnimPos() == 0:
        dirs = ["L", "R", "U", "D"]
        # boolean to keep from abruptly turning
        changeDir = 0
        options = []
        for d in dirs:
            if noWall(d,maze,sprite): 
                options.append(d)
                if d != direct and d != getOppositeDir(direct):
                    changeDir = 1
        if changeDir and '*' in maze:
            pacmanpos = np.where(maze=='*')
            pacmanx = pacmanpos[1][0]
            pacmany = pacmanpos[0][0]
            direct = getBlinkyMove(sprite,options, pacmanx,pacmany,maze)
        else:
            if not noWall(direct,maze,sprite):
                direct = getOppositeDir(direct)
    sprite.updateAnimPos(direct,maze)

In [12]:
# returns a tuple with the first
# element as the state and the
# second as the move
def stateMoveTuple(pacman,ghosts,move,symbol):
    t = "P"+str(pacman.getPosition())
    for g in ghosts:
        t += " " + g.getSymbol() + str(g.getPosition())
    t += " " + move
    t += " " + symbol
    return t

In [13]:
def getTile(sprite,maze,m):
        if m == 'L':
            return maze[sprite.posY][sprite.posX - 1]
        if m == 'R':
            return maze[sprite.posY][sprite.posX + 1]
            self.posX += 1
        if m == 'U':
            return maze[sprite.posY - 1][sprite.posX]
        if m == 'D':
            return maze[sprite.posY + 1][sprite.posX]   

In [14]:
def QMove(sprite, maze, Q, epsilon, ghosts):
    direct = sprite.getDir()
    if (sprite.getAnimPos() == 0):
        if np.random.uniform() < epsilon:
            validMoves = getValidMoves(sprite, maze,False)
            index = random.randint(0,len(validMoves)-1)
            direct = validMoves[index]
        else:
            validMoves = getValidMoves(sprite, maze,False)
            Qs = np.array([Q.get(stateMoveTuple(sprite,ghosts,m, getTile(sprite,maze,m)), 0) for m in validMoves])
            direct = validMoves[np.argmax(Qs)]
    if stateMoveTuple(sprite, ghosts,direct,getTile(sprite,maze,direct)) not in Q:
        Q[stateMoveTuple(sprite,ghosts,direct,getTile(sprite,maze,direct))] = 0
    key = stateMoveTuple(sprite,ghosts,direct,getTile(sprite,maze,direct))
    sprite.updateAnimPos(direct,maze)
    
    return Q, direct, key

In [15]:
def initMaze(mazename,size, smartghosts,numghosts,ghostspositions):
    maze = np.genfromtxt(mazename +".txt", dtype="str", delimiter=" ")
    displaywidth = size * len(maze[0])
    displayheight = size * len(maze)
    background = pygame.image.load(mazename +".png")
    pacmanimages = []
    pacmanimages.append("pacman-closed.png")
    pacmanimages.append("pacman-open-down.png")
    pacmanimages.append("pacman-open-left.png")
    pacmanimages.append("pacman-open-right.png")
    pacmanimages.append("pacman-open-up.png")
    pacmanimages.append("pacman-wide-down.png")
    pacmanimages.append("pacman-wide-left.png")
    pacmanimages.append("pacman-wide-right.png")
    pacmanimages.append("pacman-wide-up.png")
    pacmananims = {}
    pacmananims["D"] = [0,1,5,1]
    pacmananims["L"] = [0,2,6,2]
    pacmananims["R"] = [0,3,7,3]
    pacmananims["U"] = [0,4,8,4]
    pacmansymbol = '*'
    pacmanpos = np.where(maze==pacmansymbol)
    pacmanx = pacmanpos[1][0]
    pacmany = pacmanpos[0][0]
    if useQ:
        moveF = QMove
    else:
        moveF = randomMove
    pacman = MovingObject(pacmanimages, pacmanx, pacmany, moveF, pacmananims, len(pacmananims),
          pacmansymbol, 4, size, 'L')
    ghosts = []
    ghostsymbols = ['B','P','I','C']
    ghostnames = ['blinky','pinky', 'inky', 'clyde']
    for i in range(numghosts):
        ghostimages = [ghostnames[i]+"-down.png",ghostnames[i]+"-left.png",ghostnames[i]+"-right.png",
            ghostnames[i]+"-up.png", "frightened-blue.png", "frightened-white.png"]
        ghostanims = {}
        ghostanims["D"] = [0]
        ghostanims["L"] = [1]
        ghostanims["R"] = [2]
        ghostanims["U"] = [3]
        ghostanims["F"] = [4,5]
        ghostpos = np.where(maze==ghostsymbols[i])
        ghostx = ghostspositions[i][1]
        ghosty = ghostspositions[i][0]
        if smartghosts:
            moveF = blinkyMove
        else:
            moveF = randomMove
        ghosts.append(MovingObject(ghostimages, ghostx, ghosty, moveF,
            ghostanims, 4, ghostsymbols[i], 4, size, 'L'))
            
    return maze, pacman, ghosts, background, displaywidth, displayheight

maze, pacman, ghosts, background, displaywidth, displayheight = initMaze("small-maze",24, True,1,[(6,7)])
print(pacman.getPosition())

(8, 12)


In [16]:
def showMaze(maze, screen, size):
    for i in range(len(maze)):
        for j in range(len(maze[i])):
            if maze[i][j]=='.':    
                screen.blit(pygame.image.load("pellet.png"),(j*size+8,i*size+8)) 
            if maze[i][j]=='$':    
                screen.blit(pygame.image.load("power-pellet.png"),(j*size-4,i*size-4))

In [17]:
def hasCollision(sprite,ghost):
    if sprite.posX == ghost.posX and sprite.posY == ghost.posY:
        return True
    if sprite.getAnimPos() >= 2 and ghost.getAnimPos() >= 2 and getOppositeDir(sprite.getDir()) == ghost.getDir():
        if sprite.getDir() == "L" and sprite.posY == ghost.posY and sprite.posX == ghost.posX - 1:
            return True
        if sprite.getDir() == "R" and sprite.posY == ghost.posY and sprite.posX - 1 == ghost.posX:
            return True
        if sprite.getDir() == "U" and sprite.posX == ghost.posX and sprite.posY - 1 == ghost.posY:
            return True
        if sprite.getDir() == "D" and sprite.posX == ghost.posX and sprite.posY == ghost.posY -1:
            return True
            
    return False

In [18]:
def updateMap(sprite, maze, ghosts):
    result = 0
    symbol = '_'
    if sprite.prevX != -1 and sprite.prevY != -1: 
        if maze[sprite.prevY][sprite.prevX] == '.':
            result += 0.01
        maze[sprite.prevY][sprite.prevX] = sprite.getSpaceSymbol()
    symbol = maze[sprite.posY][sprite.posX]
    maze[sprite.posY][sprite.posX] = '*'
    if not '.' in maze:
        return True,0.01, symbol
    for g in ghosts:
        if hasCollision(sprite, g):
            return True,-1, symbol
    return False,result, symbol

In [19]:
def checkMove(sprite,maze,ghosts):
    if not '.' in maze:
        return 1
    for g in ghosts:
        if hasCollision(sprite,g):
            return -1 
    return 0

In [20]:
def countQZeros(Q):
    count = 0
    for i in Q:
        if i == 0:
            count += 1
    return count

In [21]:
def printNonZero(Q):
    keys = list(Q)
    for i in range(len(keys)):
        if Q[keys[i]] != 0:
            print(keys[i], Q[keys[i]])

In [22]:
def printQ(Q):
    keys = list(Q)
    for i in range(len(keys)):
        print(keys[i], Q[keys[i]])

In [23]:
Q = {}
pygame.init()
numGames = 100000
epsilonDecayFactor = 0.95
learningRate = 0.75
mazename = "small-maze"
# around 20 for normal speed, 100+ for training or turn graphics off
gamespeed = 100
debug = False
size = 24
smartghosts = True
graphics = True
numghosts = 1
gpositions = [(6,7)]
showRate = 10

def playGame(Q,epsilonDecayFactor,learningRate,mazename,gamespeed,numGames,debug,graphics,smartghosts,numghosts,gpositions,showRate):
    epsilon = 1.0
    curgame = 1
    numL = 0
    numW = 0
    for i in range(numGames):
        totalScore = 0
        moves = []
        epsilon *= epsilonDecayFactor
        if debug:
            print(epsilon)
        maze, pacman, ghosts, background, displaywidth, displayheight = initMaze(mazename,size,smartghosts,numghosts,gpositions)        
        if graphics:
            screen = pygame.display.set_mode((displaywidth, displayheight))
            pygame.display.set_caption('Pacman! Game: ' + str(curgame))
            clock = pygame.time.Clock()
        gameOver = 0
        score = 0
        step = 0
        if debug:
            printMaze(maze)
        while not gameOver:           
            if graphics:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        gameOver = 1
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_ESCAPE:
                            pygame.quit()
                            sys.exit
                            gameOver = 1
            if not gameOver:
                score = pacman.getScore()
                if graphics:
                    pygame.display.set_caption('Pacman! Game: ' + str(curgame) + " Epsilon: " + str(epsilon))
                    screen.blit(background,(0,0))
                    
                if useQ:
                    Q, move, key = pacman.move(maze, Q, epsilon, ghosts)
                else:
                    pacman.move(maze)
                [x.move(maze) for x in ghosts] 
                gameOver,result, symbol = updateMap(pacman, maze, ghosts)
                if useQ:
                    if result < 0:
                        Q[key] += learningRate * (-1 - Q[key])
                    else: 
                        Q[key] = result
                        
                    if step > 0 and stateMoveTuple(prevpacman,prevghosts,moveOld,prevSymbol) in Q.keys():
                        Q[stateMoveTuple(prevpacman,prevghosts,moveOld, prevSymbol)] += learningRate * (Q[key] - Q[stateMoveTuple(prevpacman,prevghosts,moveOld,prevSymbol)])
                    moveOld = move
                    prevpacman = copy.deepcopy(pacman)
                    prevghosts = copy.deepcopy(ghosts)
                    prevSymbol = symbol
                    step += 1
                if gameOver and result > 0:
                    numW += 1
                elif result == -1:
                    numL += 1
                if graphics:
                    showMaze(maze,screen,size)
                    screen.blit(pygame.image.load(pacman.getImage(pacman.getAnimPos())),
                        (pacman.getScreenPosition()))
                    [screen.blit(pygame.image.load(x.getImage(x.getAnimPos())),(x.getScreenPosition())) for x in ghosts]
                    pygame.display.flip()
                    clock.tick(gamespeed)
                if debug:
                    print(pacman.posX,pacman.posY)
                    print(pacman.prevX,pacman.prevY)
                    printMaze(maze)
                    print()
        # print game number every showRate games
        if not graphics and curgame % showRate == 0:
            print(curgame,numW,numL,epsilon)
        if graphics:
            pygame.quit()
        curgame += 1
    if graphics:
        sys.exit
    return numL,numW

In [24]:
if debug:
    numL, numW = playGame(Q,epsilonDecayFactor,learningRate,mazename,30,1,True,True,False,numghosts,gpositions,1)
    numL, numW = playGame(Q,1,learningRate,mazename,200,1000,False,False,smartghosts,numghosts,gpositions,1000)

In [25]:
printQ(Q)

In [26]:
numL, numW = playGame(Q,1,learningRate,mazename,200,numGames*10,False,False,smartghosts,numghosts,gpositions,1000)

1000 49 951 1.0
2000 94 1906 1.0
3000 149 2851 1.0
4000 202 3798 1.0
5000 263 4737 1.0
6000 329 5671 1.0
7000 391 6609 1.0
8000 460 7540 1.0
9000 523 8477 1.0
10000 576 9424 1.0
11000 637 10363 1.0
12000 690 11310 1.0
13000 731 12269 1.0
14000 786 13214 1.0
15000 846 14154 1.0
16000 918 15082 1.0
17000 963 16037 1.0
18000 1017 16983 1.0
19000 1080 17920 1.0
20000 1132 18868 1.0
21000 1190 19810 1.0
22000 1243 20757 1.0
23000 1287 21713 1.0
24000 1343 22657 1.0
25000 1398 23602 1.0
26000 1465 24535 1.0
27000 1518 25482 1.0
28000 1563 26437 1.0
29000 1605 27395 1.0
30000 1666 28334 1.0
31000 1716 29284 1.0
32000 1768 30232 1.0
33000 1823 31177 1.0
34000 1898 32102 1.0
35000 1962 33038 1.0
36000 2026 33974 1.0
37000 2078 34922 1.0
38000 2148 35852 1.0
39000 2199 36801 1.0
40000 2252 37748 1.0
41000 2313 38687 1.0
42000 2366 39634 1.0
43000 2414 40586 1.0
44000 2471 41529 1.0
45000 2524 42476 1.0
46000 2578 43422 1.0
47000 2636 44364 1.0
48000 2691 45309 1.0
49000 2746 46254 1.0
50000 2808

In [27]:
printQ(Q)

P(8, 12) B(7, 6) L . 0
P(7, 12) B(6, 6) L . 0.0
P(7, 12) B(7, 6) L . 0
P(6, 12) B(7, 6) L . 0.0
P(6, 12) B(8, 6) L . 0
P(5, 12) B(8, 6) U _ 0
P(5, 11) B(9, 6) U _ 0.0
P(5, 11) B(8, 6) U _ 0
P(5, 10) B(8, 6) U _ 0.0
P(5, 10) B(7, 6) U _ 0
P(5, 9) B(7, 6) U _ 0.0
P(5, 9) B(6, 6) U _ 0
P(5, 8) B(6, 6) U _ 0.0
P(5, 8) B(5, 6) U _ 0
P(5, 8) B(5, 7) U _ -0.75
P(5, 12) B(8, 6) L . 0
P(5, 12) B(9, 6) L . 0
P(4, 12) B(8, 6) L . 0
P(4, 12) B(7, 6) L . 0
P(3, 12) B(7, 6) L . 0
P(3, 12) B(6, 6) L . 0
P(2, 12) B(6, 6) L . 0
P(2, 12) B(5, 6) L . 0
P(2, 12) B(5, 7) L . 0
P(1, 12) B(5, 7) D . 0
P(1, 13) B(5, 7) D . 0
P(1, 13) B(5, 8) D . 0
P(1, 14) B(5, 8) D . 0
P(1, 14) B(5, 9) D . 0
P(1, 15) B(5, 9) R . 0
P(2, 15) B(5, 9) R . 0.0
P(2, 15) B(5, 10) R . 0
P(3, 15) B(5, 10) R . 0.0
P(3, 15) B(5, 11) R . 0
P(4, 15) B(5, 11) R . 0.0
P(4, 15) B(5, 12) R . 0
P(4, 15) B(4, 12) R . 0
P(5, 15) B(4, 12) R . 0
P(5, 15) B(3, 12) R . 0
P(6, 15) B(3, 12) R . 0
P(6, 15) B(2, 12) R . 0
P(7, 15) B(2, 12) R . 0
P(7, 1

P(1, 1) B(6, 4) R . 0
P(2, 1) B(6, 4) R . 0
P(2, 1) B(6, 3) R . 0
P(3, 1) B(6, 3) R . 0
P(3, 1) B(6, 2) R . 0
P(4, 1) B(6, 2) R . 0
P(4, 1) B(6, 1) R . -0.9990234375
P(10, 12) B(10, 9) L . 0.01
P(9, 12) B(10, 9) L _ 0
P(9, 12) B(10, 10) L _ 0
P(8, 12) B(10, 10) L _ 0
P(8, 12) B(10, 11) L _ 0.0
P(7, 12) B(10, 11) L _ 0
P(7, 12) B(10, 12) L _ 0.0
P(6, 12) B(9, 12) L _ 0.0
P(5, 12) B(8, 12) U _ 0
P(5, 11) B(8, 12) U _ 0
P(5, 11) B(7, 12) U _ 0.0
P(5, 10) B(7, 12) U _ 0
P(5, 10) B(6, 12) U _ 0.0
P(5, 9) B(6, 12) U _ 0
P(5, 9) B(5, 12) U _ 0.0
P(5, 8) B(5, 11) U _ 0.0
P(5, 7) B(5, 10) U . 0
P(5, 6) B(5, 9) L . 0
P(4, 6) B(5, 9) L . 0
P(4, 6) B(5, 8) L . 0
P(3, 6) B(5, 8) L . 0
P(3, 6) B(5, 7) L . 0
P(2, 6) B(5, 7) L . 0
P(2, 6) B(5, 6) L . 0
P(1, 6) B(4, 6) U . 0
P(1, 5) B(4, 6) U . 0
P(1, 5) B(3, 6) U . 0.0
P(1, 4) B(3, 6) U . 0
P(1, 4) B(2, 6) U . 0.0
P(1, 3) B(2, 6) U . 0
P(9, 6) B(6, 5) R _ 0.0
P(9, 6) B(6, 6) R _ 0.0
P(10, 6) B(7, 6) D _ 0
P(10, 7) B(7, 6) D _ 0
P(10, 7) B(8, 6) D _ 0


P(12, 12) B(1, 11) R _ 0
P(12, 12) B(1, 12) R _ 0
P(12, 12) B(2, 12) R _ 0
P(13, 12) B(2, 12) R _ 0
P(13, 12) B(3, 12) R _ 0
P(14, 12) B(3, 12) D _ 0
P(14, 13) B(3, 12) D _ 0
P(14, 13) B(4, 12) D _ 0
P(14, 14) B(4, 12) D _ 0
P(14, 14) B(5, 12) D _ 0
P(14, 15) B(5, 12) L _ 0
P(13, 15) B(5, 12) L _ 0.0
P(13, 15) B(6, 12) L _ 0
P(12, 15) B(6, 12) L _ 0.0
P(12, 15) B(7, 12) L _ 0
P(11, 15) B(7, 12) L _ 0.0
P(11, 15) B(8, 12) L _ 0
P(10, 15) B(8, 12) L _ 0.0
P(10, 15) B(9, 12) L _ 0
P(9, 15) B(9, 12) L _ 0
P(9, 15) B(10, 12) L _ 0
P(8, 15) B(9, 12) L _ 0
P(8, 15) B(8, 12) L _ 0
P(7, 15) B(8, 12) L _ 0
P(7, 15) B(7, 12) L _ 0
P(6, 15) B(7, 12) L _ 0
P(6, 15) B(6, 12) L _ 0
P(5, 15) B(6, 12) L _ 0
P(5, 15) B(5, 12) L _ 0
P(5, 15) B(5, 11) L _ 0
P(4, 15) B(5, 11) L _ 0
P(4, 15) B(5, 10) L _ 0
P(3, 15) B(5, 10) L _ 0
P(3, 15) B(5, 9) L _ 0
P(2, 15) B(5, 9) L _ 0
P(2, 15) B(5, 8) L _ 0
P(1, 15) B(5, 8) U _ 0
P(1, 14) B(5, 8) U _ 0
P(1, 14) B(5, 7) U _ 0
P(1, 13) B(5, 7) U _ 0
P(1, 13) B(5, 6) U 

P(14, 6) B(1, 2) D . 0
P(14, 7) B(1, 3) D . 0
P(14, 8) B(1, 4) D . 0
P(14, 9) B(1, 5) D . 0
P(14, 10) B(1, 6) D . 0
P(14, 11) B(1, 7) D _ 0
P(14, 11) B(1, 8) D _ 0
P(14, 12) B(1, 8) L _ 0
P(13, 12) B(1, 8) L _ 0
P(13, 12) B(1, 9) L _ 0
P(12, 12) B(1, 9) L _ 0
P(12, 12) B(1, 10) L _ 0
P(11, 12) B(1, 10) L _ 0
P(11, 12) B(1, 11) L _ 0
P(10, 12) B(1, 11) U _ 0
P(10, 11) B(1, 11) U _ 0
P(10, 11) B(1, 12) U _ 0
P(10, 10) B(1, 11) U _ 0
P(10, 10) B(1, 10) U _ 0
P(10, 9) B(1, 10) U _ 0
P(10, 9) B(1, 9) U _ 0
P(10, 8) B(1, 9) U _ 0
P(10, 8) B(1, 8) U _ 0
P(10, 7) B(1, 8) U . 0
P(10, 7) B(1, 7) U . 0
P(10, 6) B(1, 7) L _ 0
P(9, 6) B(1, 7) U _ 0
P(9, 5) B(1, 6) U _ 0.0
P(9, 4) B(1, 5) U _ 0.0
P(9, 3) B(1, 4) U _ 0.0
P(9, 2) B(1, 3) U _ 0.0
P(9, 1) B(1, 2) R _ 0
P(10, 1) B(1, 2) R _ 0
P(10, 1) B(1, 1) R _ 0.0
P(11, 1) B(2, 1) R _ 0.0
P(14, 1) B(5, 1) D _ 0
P(14, 2) B(5, 1) D _ 0
P(14, 2) B(6, 1) D _ 0.0
P(14, 3) B(6, 2) D _ 0.0
P(13, 6) B(14, 14) L _ 0
P(12, 6) B(14, 13) L _ 0
P(11, 6) B(14, 12) 

P(7, 12) B(10, 12) R _ 0
P(8, 12) B(9, 12) R . -0.75
P(14, 1) B(6, 3) D . 0
P(14, 4) B(6, 6) D . 0
P(14, 12) B(14, 9) U _ 0
P(5, 12) B(8, 6) L _ 0
P(2, 12) B(5, 6) L _ 0
P(10, 10) B(6, 12) U _ 0
P(10, 9) B(7, 12) U _ 0
P(10, 8) B(8, 12) U _ 0
P(10, 7) B(9, 12) U . 0
P(10, 7) B(10, 12) U . 0
P(11, 6) B(10, 11) R . 0
P(12, 6) B(10, 10) R . 0
P(13, 6) B(10, 9) R _ 0
P(14, 4) B(10, 6) U _ 0.0
P(14, 1) B(13, 6) L _ 0
P(13, 1) B(14, 6) L _ 0
P(9, 1) B(14, 2) D _ 0
P(9, 2) B(14, 1) D _ 0
P(9, 4) B(14, 4) D _ 0
P(9, 5) B(14, 5) D _ 0
P(9, 6) B(14, 5) L _ 0
P(8, 6) B(14, 6) L _ 0
P(8, 6) B(13, 6) L _ 0
P(7, 6) B(12, 6) L _ 0
P(6, 6) B(5, 10) R _ 0
P(7, 6) B(5, 9) R _ 0
P(8, 6) B(5, 8) R _ 0
P(9, 6) B(5, 7) R _ 0.0
P(1, 6) B(14, 4) R _ 0
P(3, 6) B(14, 6) R _ 0
P(4, 6) B(13, 6) R _ 0
P(10, 12) B(14, 14) L _ 0
P(1, 12) B(6, 12) U _ 0
P(1, 11) B(6, 12) U _ 0
P(6, 6) B(6, 2) R _ 0
P(9, 6) B(6, 4) R . 0.0
P(9, 6) B(6, 5) R . 0.0
P(10, 12) B(10, 8) L _ 0
P(1, 12) B(5, 12) D _ 0
P(5, 15) B(1, 15) R _ 0

P(3, 1) B(14, 3) L . 0
P(2, 1) B(14, 2) L . 0
P(1, 4) B(11, 1) D . 0
P(1, 5) B(10, 1) D . 0
P(1, 8) B(9, 4) D . 0
P(1, 9) B(9, 5) D . 0
P(14, 12) B(6, 12) L _ 0
P(10, 12) B(9, 12) L _ -0.57421875
P(1, 10) B(9, 6) D . 0.01
P(5, 6) B(10, 6) D _ 0
P(10, 12) B(10, 6) U _ 0
P(10, 12) B(7, 6) R . 0
P(12, 12) B(8, 6) R . 0
P(13, 12) B(10, 7) R _ 0
P(14, 11) B(10, 7) U _ 0.0
P(14, 10) B(10, 8) U _ 0.0
P(14, 7) B(10, 12) U _ 0.0
P(14, 6) B(10, 11) U _ 0.0
P(14, 5) B(10, 10) U _ 0.0
P(14, 4) B(10, 9) U _ 0.0
P(14, 3) B(10, 8) U _ 0.0
P(14, 2) B(10, 7) U _ 0.0
P(13, 1) B(11, 6) L _ 0.0
P(12, 1) B(12, 6) L _ 0.0
P(11, 1) B(13, 6) L _ 0.0
P(10, 6) B(6, 1) D _ 0
P(10, 11) B(6, 6) D _ 0.0
P(10, 12) B(7, 6) L _ 0
P(5, 10) B(9, 6) U _ 0
P(5, 9) B(8, 6) U _ 0
P(5, 8) B(7, 6) U _ 0
P(5, 7) B(6, 6) U _ 0
P(5, 7) B(5, 6) U _ -0.75
P(1, 11) B(1, 10) U _ -0.9990234375
P(6, 1) B(10, 1) D . 0
P(6, 2) B(10, 1) D . 0
P(6, 2) B(11, 1) D . 0
P(6, 3) B(11, 1) D . 0
P(6, 3) B(12, 1) D . 0
P(6, 4) B(12, 1) D . 0
P(6,

In [28]:
numL, numW = playGame(Q,.99999,learningRate,mazename,200,numGames,False,False,smartghosts,numghosts,gpositions,1000)

1000 46 954 0.9900497842463938
2000 108 1892 0.980198575286327
3000 171 2829 0.970445387980849
4000 224 3776 0.9607892469933452
5000 272 4728 0.9512291866920142
6000 318 5682 0.9417642510533004
7000 375 6625 0.932393493566284
8000 427 7573 0.9231159771380392
9000 479 8521 0.9139307739999132
10000 527 9473 0.9048369656147537


In [29]:
print("Number of Wins: " + str(numW))
print("Number of Losses: " + str(numL))

Number of Wins: 527
Number of Losses: 9473


In [30]:
printQ(Q)

P(8, 12) B(7, 6) L . 0
P(7, 12) B(6, 6) L . 0.0
P(7, 12) B(7, 6) L . 0
P(6, 12) B(7, 6) L . 0.0
P(6, 12) B(8, 6) L . 0
P(5, 12) B(8, 6) U _ 0
P(5, 11) B(9, 6) U _ 0.0
P(5, 11) B(8, 6) U _ 0
P(5, 10) B(8, 6) U _ 0.0
P(5, 10) B(7, 6) U _ 0
P(5, 9) B(7, 6) U _ 0.0
P(5, 9) B(6, 6) U _ 0
P(5, 8) B(6, 6) U _ 0.0
P(5, 8) B(5, 6) U _ 0
P(5, 8) B(5, 7) U _ -0.75
P(5, 12) B(8, 6) L . 0
P(5, 12) B(9, 6) L . 0
P(4, 12) B(8, 6) L . 0
P(4, 12) B(7, 6) L . 0
P(3, 12) B(7, 6) L . 0
P(3, 12) B(6, 6) L . 0
P(2, 12) B(6, 6) L . 0
P(2, 12) B(5, 6) L . 0
P(2, 12) B(5, 7) L . 0
P(1, 12) B(5, 7) D . 0
P(1, 13) B(5, 7) D . 0
P(1, 13) B(5, 8) D . 0
P(1, 14) B(5, 8) D . 0
P(1, 14) B(5, 9) D . 0
P(1, 15) B(5, 9) R . 0
P(2, 15) B(5, 9) R . 0.0
P(2, 15) B(5, 10) R . 0
P(3, 15) B(5, 10) R . 0.0
P(3, 15) B(5, 11) R . 0
P(4, 15) B(5, 11) R . 0.0
P(4, 15) B(5, 12) R . 0
P(4, 15) B(4, 12) R . 0
P(5, 15) B(4, 12) R . 0
P(5, 15) B(3, 12) R . 0
P(6, 15) B(3, 12) R . 0
P(6, 15) B(2, 12) R . 0
P(7, 15) B(2, 12) R . 0
P(7, 1

P(12, 6) B(7, 6) L . 0
P(12, 6) B(8, 6) L . 0
P(11, 6) B(8, 6) L . 0
P(11, 6) B(9, 6) L . 0
P(10, 6) B(9, 6) L _ -0.75
P(14, 6) B(14, 9) U . 0
P(14, 5) B(14, 8) U . 0
P(14, 4) B(14, 7) U . 0
P(14, 3) B(14, 6) U . 0
P(14, 2) B(14, 5) U . 0
P(14, 1) B(14, 4) L . 0
P(13, 1) B(14, 4) L . 0
P(13, 1) B(14, 3) L . 0
P(12, 1) B(14, 3) L . 0
P(12, 1) B(14, 2) L . 0
P(11, 1) B(14, 2) L . 0
P(11, 1) B(14, 1) L . 0
P(10, 1) B(13, 1) L . 0
P(9, 1) B(12, 1) D . 0
P(9, 2) B(12, 1) D . 0
P(9, 2) B(11, 1) D . 0
P(9, 3) B(11, 1) D . 0
P(9, 3) B(10, 1) D . 0
P(9, 4) B(10, 1) D . 0
P(9, 4) B(9, 1) D . 0
P(9, 5) B(9, 2) D . 0
P(9, 6) B(9, 3) R _ 0
P(10, 6) B(9, 3) R _ 0
P(10, 6) B(9, 4) R _ 0
P(11, 6) B(9, 4) R _ 0
P(11, 6) B(9, 5) R _ 0
P(12, 6) B(9, 5) R _ 0
P(12, 6) B(9, 6) R _ 0.0
P(13, 6) B(10, 6) R _ 0.0
P(14, 6) B(11, 6) U _ 0
P(14, 5) B(11, 6) U _ 0
P(14, 5) B(12, 6) U _ 0
P(14, 4) B(12, 6) U _ 0
P(14, 4) B(13, 6) U _ 0
P(14, 3) B(13, 6) U _ 0
P(14, 3) B(14, 6) U _ 0
P(14, 2) B(14, 5) U _ 0
P(14, 1

P(14, 1) B(4, 1) D _ 0
P(14, 2) B(3, 1) D _ 0
P(14, 3) B(2, 1) D _ 0
P(14, 4) B(1, 1) D _ 0
P(14, 4) B(1, 2) D _ 0
P(14, 5) B(1, 2) D _ 0
P(14, 5) B(1, 3) D _ 0
P(14, 6) B(1, 3) L . 0
P(13, 6) B(1, 4) L . 0
P(12, 6) B(1, 5) L . 0
P(6, 6) B(9, 5) U . 0
P(6, 5) B(9, 5) U . 0
P(6, 5) B(9, 6) U . 0
P(6, 4) B(9, 5) U . 0
P(6, 4) B(9, 4) U . 0
P(6, 3) B(9, 4) U . 0
P(6, 3) B(9, 3) U . 0
P(6, 2) B(9, 3) U . 0
P(6, 2) B(9, 2) U . 0
P(6, 1) B(9, 2) L . 0
P(5, 1) B(9, 2) L . 0
P(5, 1) B(9, 1) L . 0
P(3, 1) B(9, 4) L . 0
P(2, 1) B(9, 4) L . 0
P(2, 1) B(9, 5) L . 0
P(1, 1) B(9, 5) D . 0
P(1, 2) B(9, 5) D . 0
P(1, 2) B(9, 6) D . 0
P(1, 6) B(9, 2) R . 0
P(2, 6) B(9, 2) R . 0
P(2, 6) B(9, 1) R . 0
P(4, 6) B(9, 4) R . 0
P(5, 6) B(9, 4) R _ 0.0
P(5, 6) B(9, 5) R _ 0
P(6, 6) B(9, 5) U _ 0
P(6, 5) B(9, 5) U _ 0
P(6, 5) B(9, 6) U _ 0
P(6, 4) B(9, 5) U _ 0
P(6, 4) B(9, 4) U _ 0
P(6, 3) B(9, 4) U _ 0
P(6, 3) B(9, 3) U _ 0
P(6, 2) B(9, 3) U _ 0
P(6, 2) B(9, 2) U _ 0
P(6, 1) B(9, 2) L _ 0
P(5, 1) B(9, 2) L _ 

P(1, 8) B(10, 10) U _ 0
P(1, 8) B(10, 9) U _ 0
P(1, 7) B(10, 9) U _ 0
P(1, 7) B(10, 8) U _ 0
P(1, 6) B(10, 8) U _ 0.0
P(1, 6) B(10, 7) U _ 0
P(1, 5) B(10, 7) U _ 0.0
P(1, 5) B(10, 6) U _ 0
P(1, 5) B(9, 5) U _ 0
P(1, 4) B(9, 5) U _ 0
P(1, 4) B(9, 4) U _ 0
P(1, 3) B(9, 4) U _ 0
P(1, 3) B(9, 3) U _ 0
P(1, 2) B(9, 3) U _ 0
P(1, 2) B(9, 2) U _ 0
P(1, 1) B(9, 2) R _ 0
P(2, 1) B(9, 2) R _ 0
P(2, 1) B(9, 1) R _ 0
P(3, 1) B(9, 2) R _ 0
P(3, 1) B(9, 3) R _ 0
P(4, 1) B(9, 3) R _ 0
P(4, 1) B(9, 4) R _ 0
P(5, 1) B(9, 4) R _ 0
P(5, 1) B(9, 5) R _ 0
P(6, 1) B(9, 5) D _ 0
P(6, 2) B(9, 5) D _ 0
P(6, 2) B(9, 6) D _ 0
P(6, 3) B(9, 5) D _ 0
P(6, 3) B(9, 4) D _ 0
P(6, 4) B(9, 4) D _ 0
P(6, 4) B(9, 3) D _ 0
P(6, 5) B(9, 3) D _ 0
P(6, 5) B(9, 2) D _ 0
P(7, 6) B(9, 2) R _ 0
P(7, 6) B(9, 1) R _ 0
P(8, 6) B(9, 2) R _ 0
P(8, 6) B(9, 3) R _ 0
P(9, 6) B(9, 3) R . 0
P(9, 6) B(9, 4) R . 0
P(14, 12) B(12, 12) D . 0
P(14, 13) B(12, 12) D . 0
P(14, 13) B(13, 12) D . 0
P(14, 14) B(13, 12) D . 0
P(14, 14) B(14, 12) D . 0

P(14, 6) B(1, 1) D . 0
P(14, 7) B(1, 2) D . 0.0
P(14, 8) B(1, 3) D . 0.0
P(14, 9) B(1, 4) D . 0.0
P(14, 10) B(1, 5) D . 0.0
P(14, 11) B(1, 6) D _ 0
P(14, 12) B(1, 7) D _ 0.0
P(14, 13) B(1, 8) D _ 0.0
P(14, 14) B(1, 9) D _ 0.0
P(14, 15) B(1, 10) L _ 0
P(9, 15) B(1, 15) L _ 0.0
P(8, 15) B(2, 15) L _ 0.0
P(7, 15) B(3, 15) L _ 0.0
P(6, 15) B(4, 15) L _ 0.0
P(6, 6) B(1, 2) R . 0
P(6, 6) B(10, 6) R _ 0.0
P(6, 6) B(9, 4) L . 0.0
P(8, 12) B(5, 10) R _ 0
P(10, 12) B(6, 12) R _ 0.0
P(11, 12) B(7, 12) R _ 0
P(12, 12) B(8, 12) R _ 0
P(13, 12) B(9, 12) R _ 0
P(14, 9) B(10, 9) U _ 0
P(14, 8) B(10, 8) U _ 0.0
P(14, 7) B(10, 7) U _ 0
P(14, 6) B(10, 7) L _ 0
P(13, 6) B(10, 6) L _ 0
P(13, 6) B(11, 6) L _ 0
P(11, 15) B(10, 12) R _ 0
P(12, 15) B(11, 12) R _ 0
P(13, 15) B(12, 12) R _ 0
P(14, 15) B(13, 12) U _ 0
P(14, 14) B(13, 12) U _ 0
P(14, 14) B(14, 12) U _ 0
P(9, 12) B(6, 12) R . 0.0
P(5, 6) B(9, 2) L . 0
P(5, 6) B(9, 1) L . 0
P(4, 6) B(9, 2) L . 0
P(4, 6) B(9, 3) L . 0
P(3, 6) B(9, 3) L . 0
P(3, 6) B(

P(9, 2) B(1, 1) D _ 0.0
P(9, 2) B(1, 2) D _ 0
P(9, 3) B(1, 2) D _ 0.0
P(9, 3) B(1, 3) D _ 0
P(9, 4) B(1, 3) D _ 0.0
P(9, 4) B(1, 4) D _ 0
P(10, 6) B(1, 6) R _ 0
P(11, 6) B(2, 6) R _ 0
P(12, 6) B(3, 6) R _ 0
P(13, 6) B(4, 6) R _ 0
P(14, 6) B(5, 6) D _ 0
P(14, 11) B(10, 6) D _ 0
P(14, 12) B(10, 7) D _ 0
P(14, 13) B(10, 8) D _ 0
P(14, 14) B(10, 9) D _ 0
P(14, 15) B(10, 10) L _ 0
P(6, 6) B(11, 6) L . 0
P(5, 12) B(14, 10) U _ 0
P(1, 10) B(9, 12) U _ 0
P(1, 9) B(8, 12) U _ 0
P(1, 8) B(7, 12) U _ 0
P(1, 7) B(6, 12) U _ 0
P(9, 6) B(1, 8) U . 0
P(14, 8) B(6, 6) D . 0
P(14, 6) B(5, 8) U . 0.0
P(14, 5) B(5, 7) U . 0.0
P(9, 2) B(2, 1) D _ 0
P(6, 6) B(3, 6) L . 0
P(1, 12) B(10, 10) R _ 0
P(3, 12) B(10, 12) R _ 0.0
P(4, 12) B(9, 12) R _ 0.0
P(6, 6) B(14, 8) U . 0
P(6, 6) B(10, 7) U . 0
P(14, 1) B(14, 7) L _ 0
P(5, 6) B(2, 6) L . 0
P(4, 6) B(3, 6) L . -0.9999847412109375
P(6, 6) B(14, 9) U _ 0
P(5, 1) B(14, 4) L _ 0
P(4, 1) B(14, 3) L _ 0
P(3, 1) B(14, 2) L _ 0
P(1, 4) B(10, 1) D _ 0
P(9, 12) B(9, 6)

In [31]:
numL, numW = playGame(Q,.99,learningRate,mazename,200,1000,False,False,smartghosts,numghosts,gpositions,showRate)

10 0 10 0.9043820750088043
20 0 20 0.8179069375972307
30 0 30 0.7397003733882802
40 0 40 0.6689717585696803
50 0 50 0.6050060671375365
60 1 59 0.5471566423907612
70 1 69 0.49483865960020695
80 1 79 0.44752321376381066
90 1 89 0.4047319726783239
100 2 98 0.36603234127322926
110 2 108 0.33103308832101386
120 2 118 0.29938039131233124
130 2 128 0.270754259511994
140 2 138 0.24486529903492946
150 2 148 0.22145178723886094
160 2 158 0.20027702685748935
170 2 168 0.18112695312597027
180 2 178 0.16380796970808745
190 2 188 0.1481449915475795
200 2 198 0.13397967485796175
210 2 208 0.1211688163570484
220 2 218 0.10958290556334822
230 2 228 0.09910481551887473
240 2 238 0.08962861870232469
250 2 248 0.08105851616218133
260 2 258 0.07330786904388827
270 2 268 0.06629832272038537
280 2 278 0.05995901467146548
290 2 288 0.054225858104063294
300 2 298 0.04904089407128576
310 2 308 0.04435170554047638
320 2 318 0.04011088748687551
330 2 328 0.036275567655825146
340 2 338 0.03280697314869741
350 2 34

In [32]:
print("Number of Wins: " + str(numW))
print("Number of Losses: " + str(numL))

Number of Wins: 2
Number of Losses: 998


In [33]:
printQ(Q)

P(8, 12) B(7, 6) L . 0
P(7, 12) B(6, 6) L . 0.0
P(7, 12) B(7, 6) L . 0
P(6, 12) B(7, 6) L . 0.0
P(6, 12) B(8, 6) L . 0
P(5, 12) B(8, 6) U _ 0
P(5, 11) B(9, 6) U _ 0.0
P(5, 11) B(8, 6) U _ 0
P(5, 10) B(8, 6) U _ 0.0
P(5, 10) B(7, 6) U _ 0
P(5, 9) B(7, 6) U _ 0.0
P(5, 9) B(6, 6) U _ 0
P(5, 8) B(6, 6) U _ 0.0
P(5, 8) B(5, 6) U _ 0
P(5, 8) B(5, 7) U _ -0.75
P(5, 12) B(8, 6) L . 0
P(5, 12) B(9, 6) L . 0
P(4, 12) B(8, 6) L . 0
P(4, 12) B(7, 6) L . 0
P(3, 12) B(7, 6) L . 0
P(3, 12) B(6, 6) L . 0
P(2, 12) B(6, 6) L . 0
P(2, 12) B(5, 6) L . 0
P(2, 12) B(5, 7) L . 0
P(1, 12) B(5, 7) D . 0
P(1, 13) B(5, 7) D . 0
P(1, 13) B(5, 8) D . 0
P(1, 14) B(5, 8) D . 0
P(1, 14) B(5, 9) D . 0
P(1, 15) B(5, 9) R . 0
P(2, 15) B(5, 9) R . 0.0
P(2, 15) B(5, 10) R . 0
P(3, 15) B(5, 10) R . 0.0
P(3, 15) B(5, 11) R . 0
P(4, 15) B(5, 11) R . 0.0
P(4, 15) B(5, 12) R . 0
P(4, 15) B(4, 12) R . 0
P(5, 15) B(4, 12) R . 0
P(5, 15) B(3, 12) R . 0
P(6, 15) B(3, 12) R . 0
P(6, 15) B(2, 12) R . 0
P(7, 15) B(2, 12) R . 0
P(7, 1

P(11, 1) B(14, 2) L . 0
P(11, 1) B(14, 1) L . 0
P(10, 1) B(13, 1) L . 0
P(9, 1) B(12, 1) D . 0
P(9, 2) B(12, 1) D . 0
P(9, 2) B(11, 1) D . 0
P(9, 3) B(11, 1) D . 0
P(9, 3) B(10, 1) D . 0
P(9, 4) B(10, 1) D . 0
P(9, 4) B(9, 1) D . 0
P(9, 5) B(9, 2) D . 0.0
P(9, 6) B(9, 3) R _ 0
P(10, 6) B(9, 3) R _ 0.0
P(10, 6) B(9, 4) R _ 0
P(11, 6) B(9, 4) R _ 0
P(11, 6) B(9, 5) R _ 0
P(12, 6) B(9, 5) R _ 0
P(12, 6) B(9, 6) R _ 0
P(13, 6) B(10, 6) R _ 0
P(14, 6) B(11, 6) U _ 0
P(14, 5) B(11, 6) U _ 0
P(14, 5) B(12, 6) U _ 0
P(14, 4) B(12, 6) U _ 0
P(14, 4) B(13, 6) U _ 0
P(14, 3) B(13, 6) U _ 0
P(14, 3) B(14, 6) U _ 0
P(14, 2) B(14, 5) U _ 0
P(14, 1) B(14, 4) L _ 0
P(13, 1) B(14, 4) L _ 0
P(13, 1) B(14, 3) L _ 0
P(12, 1) B(14, 3) L _ 0
P(12, 1) B(14, 2) L _ 0
P(11, 1) B(14, 2) L _ 0
P(11, 1) B(14, 1) L _ 0
P(10, 1) B(13, 1) L _ 0
P(9, 1) B(12, 1) D _ 0
P(9, 2) B(12, 1) D _ 0
P(9, 2) B(11, 1) D _ 0
P(9, 3) B(11, 1) D _ 0
P(9, 3) B(10, 1) D _ 0
P(9, 4) B(10, 1) D _ 0
P(9, 4) B(9, 1) D _ 0
P(9, 5) B(9, 2

P(4, 6) B(14, 6) R _ 0
P(1, 7) B(5, 11) U _ 0.0
P(1, 6) B(5, 10) R _ 0
P(2, 6) B(5, 10) R _ 0
P(2, 6) B(5, 9) R _ 0
P(3, 6) B(5, 9) R _ 0
P(3, 6) B(5, 8) R _ 0
P(4, 6) B(5, 8) R _ 0
P(4, 6) B(5, 7) R _ -0.75
P(5, 6) B(5, 7) R _ 0
P(1, 12) B(5, 15) R _ 0
P(2, 12) B(4, 15) R _ 0
P(3, 12) B(3, 15) R _ 0
P(4, 12) B(2, 15) R _ 0
P(5, 12) B(1, 15) R _ 0
P(5, 12) B(1, 14) R _ 0
P(6, 12) B(1, 13) R _ 0
P(7, 12) B(1, 12) R _ 0
P(7, 12) B(2, 12) R _ 0
P(8, 12) B(3, 12) R . 0
P(9, 12) B(4, 12) R . 0
P(10, 12) B(4, 12) U _ 0
P(10, 11) B(5, 12) U _ 0
P(10, 11) B(5, 11) U _ 0
P(10, 10) B(5, 10) U _ 0
P(10, 9) B(5, 9) U _ 0
P(10, 8) B(5, 8) U _ 0
P(10, 7) B(5, 8) U . 0
P(10, 7) B(5, 7) U . 0
P(10, 6) B(5, 7) L _ 0
P(9, 6) B(5, 6) L _ 0.0
P(9, 6) B(6, 6) L _ 0
P(8, 6) B(7, 6) L _ -0.984375
P(14, 5) B(6, 4) D _ 0
P(14, 5) B(6, 5) D _ 0
P(14, 6) B(6, 5) L _ 0
P(13, 6) B(6, 5) L _ 0
P(13, 6) B(6, 6) L _ 0
P(13, 6) B(7, 6) L _ 0
P(12, 6) B(7, 6) L _ 0
P(12, 6) B(8, 6) L _ 0
P(11, 6) B(8, 6) L _ 0
P(11, 6)

P(1, 5) B(10, 6) U _ 0
P(1, 5) B(9, 5) U _ 0
P(1, 4) B(9, 5) U _ 0
P(1, 4) B(9, 4) U _ 0
P(1, 3) B(9, 4) U _ 0
P(1, 3) B(9, 3) U _ 0
P(1, 2) B(9, 3) U _ 0
P(1, 2) B(9, 2) U _ 0
P(1, 1) B(9, 2) R _ 0
P(2, 1) B(9, 2) R _ 0
P(2, 1) B(9, 1) R _ 0
P(3, 1) B(9, 2) R _ 0.0
P(3, 1) B(9, 3) R _ 0
P(4, 1) B(9, 3) R _ 0
P(4, 1) B(9, 4) R _ 0
P(5, 1) B(9, 4) R _ 0
P(5, 1) B(9, 5) R _ 0
P(6, 1) B(9, 5) D _ 0
P(6, 2) B(9, 5) D _ 0
P(6, 2) B(9, 6) D _ 0
P(6, 3) B(9, 5) D _ 0
P(6, 3) B(9, 4) D _ 0
P(6, 4) B(9, 4) D _ 0
P(6, 4) B(9, 3) D _ 0
P(6, 5) B(9, 3) D _ 0
P(6, 5) B(9, 2) D _ 0
P(7, 6) B(9, 2) R _ 0
P(7, 6) B(9, 1) R _ 0
P(8, 6) B(9, 2) R _ 0
P(8, 6) B(9, 3) R _ 0
P(9, 6) B(9, 3) R . 0
P(9, 6) B(9, 4) R . 0
P(14, 12) B(12, 12) D . 0
P(14, 13) B(12, 12) D . 0
P(14, 13) B(13, 12) D . 0
P(14, 14) B(13, 12) D . 0
P(14, 14) B(14, 12) D . 0
P(14, 15) B(14, 13) L . 0
P(13, 15) B(14, 13) L . 0
P(13, 15) B(14, 14) L . 0
P(12, 15) B(14, 14) L . 0
P(12, 15) B(14, 15) L . 0
P(11, 15) B(13, 15) L . 0
P(10, 1

P(14, 1) B(14, 4) D . 0
P(14, 2) B(14, 4) D . 0.0
P(6, 6) B(1, 12) R _ 0.0
P(6, 6) B(1, 11) R _ 0.0
P(7, 6) B(1, 11) R _ 0.0
P(7, 6) B(1, 10) R _ 0
P(8, 6) B(1, 10) R _ 0.0
P(8, 6) B(1, 9) R _ 0
P(9, 6) B(1, 9) U . 0
P(9, 5) B(1, 8) U . 0
P(9, 4) B(1, 7) U . 0
P(9, 3) B(1, 6) U . 0
P(9, 2) B(1, 5) U . 0
P(9, 1) B(1, 4) R . 0
P(10, 1) B(1, 4) R . 0
P(10, 1) B(1, 3) R . 0
P(11, 1) B(1, 3) R . 0
P(11, 1) B(1, 2) R . 0
P(12, 1) B(1, 2) R . 0
P(12, 1) B(1, 1) R . 0
P(14, 3) B(5, 1) D . 0
P(14, 4) B(5, 1) D . 0
P(14, 4) B(6, 1) D . 0
P(14, 6) B(6, 3) L . 0
P(13, 6) B(6, 3) L . 0.0
P(12, 6) B(6, 5) L . 0
P(11, 6) B(6, 5) L _ 0
P(1, 4) B(4, 1) D _ 0
P(1, 4) B(5, 1) D _ 0
P(1, 5) B(5, 1) D _ 0
P(1, 5) B(6, 1) D _ 0
P(1, 6) B(6, 2) D _ 0.0
P(1, 7) B(6, 3) D _ 0
P(1, 8) B(6, 4) D _ 0
P(1, 9) B(6, 5) D _ 0
P(10, 12) B(14, 8) L _ 0
P(1, 3) B(5, 6) U . 0
P(1, 1) B(3, 6) R . 0
P(3, 1) B(1, 6) R . 0
P(6, 1) B(1, 3) D . 0
P(6, 3) B(1, 1) D . 0
P(6, 5) B(1, 4) D . 0
P(6, 6) B(1, 4) R . 0
P(7, 6) B(1, 5)

P(2, 15) B(14, 12) R . 0
P(3, 15) B(14, 13) R . 0
P(4, 15) B(14, 14) R . 0
P(5, 15) B(14, 15) R . 0
P(5, 15) B(13, 15) R . 0
P(6, 15) B(12, 15) R . 0
P(7, 15) B(11, 15) R . 0
P(8, 15) B(10, 15) R . 0
P(14, 6) B(4, 6) D . 0
P(14, 7) B(4, 6) D . 0.0
P(14, 7) B(5, 6) D . 0
P(14, 7) B(5, 7) D . 0
P(14, 8) B(5, 7) D . 0
P(14, 8) B(5, 8) D . 0
P(14, 9) B(5, 8) D . 0
P(14, 9) B(5, 9) D . 0
P(14, 10) B(5, 9) D . 0
P(14, 10) B(5, 10) D . 0
P(14, 11) B(5, 10) D _ 0.0
P(14, 11) B(5, 11) D _ 0
P(14, 12) B(5, 11) D _ 0.0
P(14, 6) B(6, 4) L _ 0
P(13, 6) B(6, 4) L _ 0
P(12, 6) B(6, 5) L _ 0.0
P(12, 6) B(6, 6) L _ 0
P(1, 12) B(11, 12) D _ 0
P(6, 15) B(7, 12) R _ 0
P(7, 15) B(8, 12) R _ 0
P(8, 15) B(9, 12) R _ 0
P(9, 15) B(10, 12) R _ 0
P(12, 15) B(6, 12) R _ 0
P(13, 15) B(5, 12) R _ 0
P(14, 14) B(7, 12) U _ 0
P(14, 13) B(8, 12) U _ 0
P(14, 12) B(8, 12) L . 0
P(13, 12) B(9, 12) L . 0
P(10, 12) B(9, 15) U _ 0
P(10, 11) B(9, 15) U _ 0
P(10, 10) B(10, 15) U _ 0
P(10, 9) B(11, 15) U _ 0
P(10, 8) B(12, 15) 

In [34]:
numL, numW = playGame(Q,epsilonDecayFactor,learningRate,mazename,gamespeed,100,False,graphics,smartghosts,numghosts,gpositions,0.5)

In [35]:
print("Number of Wins: " + str(numW))
print("Number of Losses: " + str(numL))

Number of Wins: 0
Number of Losses: 100


In [36]:
numL, numW = playGame(Q,0,learningRate,mazename,gamespeed,10,False,graphics,smartghosts,numghosts,gpositions,0.5)

In [37]:
printQ(Q)

P(8, 12) B(7, 6) L . 0
P(7, 12) B(6, 6) L . 0.0
P(7, 12) B(7, 6) L . 0
P(6, 12) B(7, 6) L . 0.0
P(6, 12) B(8, 6) L . 0
P(5, 12) B(8, 6) U _ 0
P(5, 11) B(9, 6) U _ 0.0
P(5, 11) B(8, 6) U _ 0
P(5, 10) B(8, 6) U _ 0.0
P(5, 10) B(7, 6) U _ 0
P(5, 9) B(7, 6) U _ 0.0
P(5, 9) B(6, 6) U _ 0
P(5, 8) B(6, 6) U _ 0.0
P(5, 8) B(5, 6) U _ 0
P(5, 8) B(5, 7) U _ -0.75
P(5, 12) B(8, 6) L . 0
P(5, 12) B(9, 6) L . 0
P(4, 12) B(8, 6) L . 0
P(4, 12) B(7, 6) L . 0
P(3, 12) B(7, 6) L . 0
P(3, 12) B(6, 6) L . 0
P(2, 12) B(6, 6) L . 0
P(2, 12) B(5, 6) L . 0
P(2, 12) B(5, 7) L . 0
P(1, 12) B(5, 7) D . 0
P(1, 13) B(5, 7) D . 0
P(1, 13) B(5, 8) D . 0
P(1, 14) B(5, 8) D . 0
P(1, 14) B(5, 9) D . 0
P(1, 15) B(5, 9) R . 0
P(2, 15) B(5, 9) R . 0.0
P(2, 15) B(5, 10) R . 0
P(3, 15) B(5, 10) R . 0.0
P(3, 15) B(5, 11) R . 0
P(4, 15) B(5, 11) R . 0.0
P(4, 15) B(5, 12) R . 0
P(4, 15) B(4, 12) R . 0
P(5, 15) B(4, 12) R . 0
P(5, 15) B(3, 12) R . 0
P(6, 15) B(3, 12) R . 0
P(6, 15) B(2, 12) R . 0
P(7, 15) B(2, 12) R . 0
P(7, 1

P(11, 6) B(10, 9) R _ 0
P(12, 6) B(10, 8) R _ 0
P(13, 6) B(10, 7) R _ 0
P(14, 8) B(10, 8) D _ 0
P(14, 9) B(10, 9) D _ 0
P(14, 10) B(10, 10) D _ 0
P(14, 11) B(10, 11) D _ 0
P(14, 12) B(10, 11) L _ 0
P(13, 12) B(10, 11) L _ 0
P(13, 12) B(10, 12) L _ 0.0
P(13, 12) B(11, 12) L _ 0
P(12, 12) B(11, 12) L _ -0.984375
P(1, 6) B(9, 6) U . 0
P(1, 5) B(8, 6) U . 0
P(1, 4) B(8, 6) U . 0
P(1, 4) B(7, 6) U . 0
P(1, 3) B(7, 6) U . 0
P(1, 3) B(6, 6) U . 0
P(1, 2) B(6, 5) U . 0
P(1, 1) B(6, 4) R . 0
P(2, 1) B(6, 4) R . 0
P(2, 1) B(6, 3) R . 0
P(3, 1) B(6, 3) R . 0
P(3, 1) B(6, 2) R . 0
P(4, 1) B(6, 2) R . 0
P(4, 1) B(6, 1) R . -1.0
P(10, 12) B(10, 9) L . 0.01
P(9, 12) B(10, 9) L _ 0
P(9, 12) B(10, 10) L _ 0
P(8, 12) B(10, 10) L _ 0
P(8, 12) B(10, 11) L _ 0
P(7, 12) B(10, 11) L _ 0
P(7, 12) B(10, 12) L _ 0
P(6, 12) B(9, 12) L _ 0
P(5, 12) B(8, 12) U _ 0
P(5, 11) B(8, 12) U _ 0
P(5, 11) B(7, 12) U _ 0.0
P(5, 10) B(7, 12) U _ 0
P(5, 10) B(6, 12) U _ 0.0
P(5, 9) B(6, 12) U _ 0
P(5, 9) B(5, 12) U _ 0
P(5, 8

P(1, 3) B(3, 1) D _ 0
P(1, 4) B(2, 1) D _ 0
P(5, 6) B(14, 8) D _ 0
P(5, 7) B(14, 7) D _ 0
P(9, 12) B(12, 12) R _ 0
P(9, 12) B(11, 12) R _ 0
P(10, 12) B(11, 12) R _ 0.0
P(1, 6) B(1, 11) U _ 0
P(1, 5) B(1, 10) U _ 0
P(1, 4) B(1, 9) U _ 0
P(1, 3) B(1, 8) U _ 0
P(1, 2) B(1, 7) U _ 0
P(1, 1) B(1, 7) R _ 0
P(2, 1) B(1, 6) R _ 0
P(3, 1) B(1, 5) R _ 0
P(4, 1) B(1, 4) R _ 0
P(5, 1) B(1, 3) R _ 0
P(6, 1) B(1, 3) D _ 0
P(6, 2) B(1, 2) D _ 0
P(6, 3) B(1, 1) D _ 0
P(6, 5) B(1, 4) D _ 0
P(6, 6) B(1, 4) R _ 0
P(7, 6) B(1, 5) R _ 0
P(8, 6) B(1, 6) R _ 0
P(8, 6) B(2, 6) R _ 0
P(9, 6) B(3, 6) R . 0
P(10, 6) B(3, 6) R . 0
P(10, 6) B(4, 6) R . 0
P(11, 6) B(4, 6) R . 0
P(11, 6) B(5, 6) R . 0
P(12, 6) B(5, 6) R . 0
P(12, 6) B(6, 6) R . 0
P(14, 6) B(7, 6) D _ 0
P(14, 7) B(7, 6) D _ 0
P(14, 7) B(8, 6) D _ 0
P(14, 8) B(8, 6) D _ 0
P(14, 8) B(9, 6) D _ 0
P(14, 9) B(9, 6) D _ 0
P(14, 9) B(10, 6) D _ 0
P(14, 9) B(10, 7) D _ 0
P(14, 10) B(10, 7) D _ 0
P(14, 10) B(10, 8) D _ 0
P(14, 11) B(10, 9) D _ 0
P(14, 12) B(1

P(7, 12) B(12, 12) R _ 0
P(8, 12) B(11, 12) R _ 0
P(1, 6) B(11, 6) U . 0
P(1, 4) B(9, 6) U . 0
P(1, 1) B(9, 3) R . 0
P(3, 1) B(9, 1) R . 0
P(4, 1) B(10, 1) R . 0
P(5, 1) B(11, 1) R . 0
P(6, 1) B(12, 1) D . 0
P(6, 2) B(12, 1) D . 0.0
P(6, 3) B(13, 1) D . 0.0
P(6, 3) B(14, 1) D . 0
P(6, 4) B(14, 2) D . 0
P(6, 5) B(14, 3) D . 0.0
P(6, 6) B(14, 4) L _ 0
P(5, 6) B(14, 4) D _ 0
P(6, 12) B(14, 10) R _ 0
P(7, 12) B(14, 11) R _ 0
P(8, 12) B(13, 12) R _ 0.0
P(9, 6) B(8, 6) U _ 0
P(14, 6) B(14, 5) L _ 0
P(14, 6) B(14, 8) L . 0
P(11, 6) B(13, 6) L . 0.0
P(10, 11) B(10, 9) D . 0
P(10, 12) B(10, 10) R . 0
P(11, 12) B(10, 10) R . 0.0
P(12, 12) B(10, 11) R . 0.0
P(9, 6) B(11, 6) U _ 0
P(6, 5) B(14, 3) D _ 0
P(6, 6) B(14, 4) R _ 0
P(7, 6) B(14, 4) R _ 0.0
P(8, 6) B(14, 5) R _ 0.0
P(9, 6) B(13, 6) U . 0
P(9, 5) B(13, 6) U . 0.0
P(9, 5) B(12, 6) U . 0
P(9, 4) B(12, 6) U . 0.0
P(9, 4) B(11, 6) U . 0
P(9, 3) B(11, 6) U . 0.0
P(9, 2) B(10, 6) U . 0.0
P(9, 2) B(9, 6) U . 0
P(9, 1) B(9, 5) R . 0
P(10, 1) B(9,

P(5, 6) B(9, 6) L . 0
P(4, 6) B(8, 6) L . 0
P(3, 6) B(7, 6) L . 0
P(9, 6) B(6, 4) U _ 0
P(9, 5) B(6, 4) U _ 0
P(14, 6) B(1, 4) L . 0
P(13, 6) B(1, 5) L . 0
P(12, 6) B(1, 6) L . 0
P(12, 6) B(2, 6) L . 0
P(5, 6) B(1, 11) L . 0
P(8, 6) B(6, 4) R . 0.0
P(9, 6) B(6, 6) R . 0.0
P(14, 8) B(6, 4) D . 0
P(14, 9) B(6, 5) D . 0
P(14, 10) B(6, 6) D . 0.01
P(14, 10) B(7, 6) D . 0.0
P(14, 12) B(9, 6) D _ 0.0
P(14, 13) B(10, 6) D _ 0
P(14, 15) B(10, 8) L _ 0
P(13, 15) B(10, 9) L _ 0.0
P(12, 15) B(10, 10) L _ 0.0
P(11, 15) B(10, 11) L _ 0.0
P(10, 15) B(10, 12) L _ 0.0
P(14, 11) B(6, 4) D . 0
P(14, 11) B(6, 5) D . 0
P(14, 12) B(6, 5) L . 0
P(13, 12) B(6, 5) L . 0
P(13, 12) B(6, 6) L . 0
P(13, 12) B(7, 6) L . 0
P(12, 12) B(7, 6) L . 0
P(12, 12) B(8, 6) L . 0
P(11, 12) B(9, 6) L _ 0
P(10, 12) B(9, 6) L _ 0
P(10, 12) B(10, 6) L _ 0
P(9, 12) B(10, 7) L _ 0
P(8, 12) B(10, 8) L _ 0
P(7, 12) B(10, 9) L _ 0
P(6, 12) B(10, 10) L _ 0
P(5, 12) B(10, 11) U _ 0
P(5, 11) B(10, 11) U _ 0
P(5, 6) B(10, 7) L . 0
P(1, 6

P(10, 6) B(1, 6) L _ 0
P(6, 6) B(10, 6) R . 0
P(6, 6) B(9, 6) R . 0
P(7, 6) B(8, 6) R . -0.9375
P(10, 12) B(9, 5) R _ 0.0
P(11, 12) B(9, 4) R _ 0
P(12, 12) B(9, 3) R _ 0
P(14, 12) B(9, 1) U . 0
P(14, 11) B(9, 2) U . 0
P(14, 10) B(9, 3) U . 0
P(14, 9) B(9, 4) U . 0
P(14, 8) B(9, 5) U . 0
P(10, 11) B(6, 5) D _ 0
P(1, 12) B(6, 1) U _ 0
P(1, 11) B(6, 2) U _ 0.0
P(1, 10) B(6, 3) U _ 0.0
P(1, 9) B(6, 4) U _ 0.0
P(1, 8) B(6, 5) U _ 0.0
P(1, 7) B(6, 6) U _ 0.0
P(1, 7) B(5, 6) U _ 0
P(1, 6) B(5, 7) R . 0
P(4, 6) B(5, 10) R . 0
P(5, 6) B(5, 10) D _ 0
P(5, 8) B(5, 12) D _ 0.0
P(5, 9) B(5, 11) D _ 0.0
P(5, 6) B(1, 12) L _ 0
P(4, 6) B(1, 12) L _ 0
P(3, 6) B(1, 11) L _ 0
P(14, 6) B(9, 6) U _ 0
P(9, 6) B(3, 6) L _ 0.0
P(8, 6) B(4, 6) L _ 0
P(14, 12) B(1, 10) L _ 0
P(12, 12) B(1, 12) L _ 0.0
P(10, 12) B(4, 12) L _ 0
P(5, 1) B(10, 1) R _ 0.0
P(6, 1) B(11, 1) D _ 0
P(6, 4) B(14, 1) D _ 0.0
P(6, 6) B(14, 3) R . 0
P(9, 6) B(14, 6) U . 0
P(9, 1) B(9, 6) R . 0
P(14, 1) B(9, 1) D . 0
P(5, 12) B(8, 12) R _ 0.